<a href="https://colab.research.google.com/github/Domdoug/Regioes-de-Saude/blob/master/Regioes_de_Saude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
nome_colunas = ['CD_MUNICIPIO', 'SG_UF', 'CD_REGIAO_SAUDE_2023', 'NM_REGIAO_SAUDE_ATUAL', 'NM_MUNICIPIO',
       'NM_REGIAO_SAUDE_ANTERIOR', 'NM_MUNICIPIOS_IN_37', 'ESTADO', 'REGIAO_PAIS']

In [3]:
df_regioes = pd.read_excel('BI_Regionais de saude_15052023.xlsx', sheet_name='Dados', skiprows=1, usecols="A:I", names=nome_colunas)

In [4]:
df_regioes.sort_values(by='SG_UF', ascending=True).head()

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
1483,120001,AC,12002,Baixo Acre e Purus,Acrelândia,Acre,Acrelândia,ACRE,NORTE
1212,120005,AC,12001,Alto Acre,Assis Brasil,Brasiléia,Assis Brasil,ACRE,NORTE
3009,120030,AC,12003,Juruá e Tarauacá/Envira,Feijó,Cruzeiro do Sul,Feijó,ACRE,NORTE
1211,120070,AC,12001,Alto Acre,Xapuri,Brasiléia,Xapuri,ACRE,NORTE
3010,120033,AC,12003,Juruá e Tarauacá/Envira,Mâncio Lima,Cruzeiro do Sul,Mâncio Lima,ACRE,NORTE


In [5]:
df_regioes.shape

(5571, 9)

In [ ]:
# INICIO DO PROCESSAMENTO

In [6]:
# Esta linha e para agilizar o groupby. Talvez desnecessaria. Lógica do do SAS
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], inplace=True)

In [7]:
# Aplicar na coluna CD_MUNICIPIO uma funcao para colocar os codigos do municipios em formato de conjunto para comparar,
# agrupados por regiao atual e UF, com o agrupamento dos municipios correspondentes da regiao anterior

lista1 = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [8]:
# Get the head and tail of the sample pandas DataFrame
# Using the pd.option_context() function in Pandas
with pd.option_context('display.max_rows',10):
    print(lista1)

NM_REGIAO_SAUDE_ATUAL         SG_UF
10ª RS Cascavel               PR       [410754, 410500, 410630, 411275, 411670, 41053...
10ª Região                    PB       [250915, 251620, 251398, 251420, 251720, 25084...
10ª Região Limoeiro do Norte  CE       [230690, 230600, 230760, 231080, 231310, 23115...
10ª Região de Saúde           AL       [270240, 270330, 270500, 270642, 270580, 27071...
11ª RS Campo Mourão           PR       [412800, 410755, 411400, 410250, 410770, 41039...
                                                             ...                        
vale do itapocú               SC       [420450, 421635, 421060, 420650, 421740, 42021...
Águas Formosas                MG       [312705, 314850, 313890, 310660, 317030, 31009...
Área Central                  AP       [160005, 160015, 160021, 160053, 160023, 16002...
Área Norte                    AP                [160070, 160010, 160050, 160020, 160055]
Área Sudoeste                 AP                        [160080, 160040, 1

In [9]:
lista1.info()

<class 'pandas.core.series.Series'>
MultiIndex: 450 entries, ('10ª RS Cascavel', 'PR') to ('Área Sudoeste', 'AP')
Series name: CD_MUNICIPIO
Non-Null Count  Dtype 
--------------  ----- 
450 non-null    object
dtypes: object(1)
memory usage: 8.6+ KB


In [10]:
# Agora a lista para a Região de Saúde anterior
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], inplace=True)

In [11]:
# Agora o código para a lista2: Regiao de Saude Anterior
lista2 = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [12]:
# Get the head and tail of the sample pandas DataFrame
# Using the pd.option_context() function in Pandas
with pd.option_context('display.max_rows',10):
    print(lista2)


NM_REGIAO_SAUDE_ANTERIOR      SG_UF
Acaraú                        CE       [230020, 230725, 230890, 230230, 230425, 23078...
Acre                          AC       [120001, 120034, 120038, 120040, 120043, 12004...
Afogados da Ingazeira         PE       [261440, 261153, 260770, 261280, 260390, 26071...
Alagoinhas                    BA       [291650, 291590, 292970, 291370, 291050, 29022...
Alfenas                       MG       [310530, 310410, 314510, 310160, 311440, 31669...
                                                             ...                        
X Núcleo Regional de Saúde    PB       [251392, 250915, 251620, 251396, 251398, 25142...
XI Núcleo Regional de Saúde   PB       [251660, 250670, 250800, 250900, 250010, 25123...
XII Núcleo Regional de Saúde  PB       [250720, 250690, 250790, 250760, 251275, 25038...
Xanxerê                       SC       [421910, 420253, 420768, 420517, 420010, 42116...
Água Boa                      MT       [510020, 510310, 510185, 510718, 51

In [13]:
# Para transformar o formato agrupado em DataFrame
lista1 = lista1.reset_index()
lista2 = lista2.reset_index()

In [14]:
lista1.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
0,10ª RS Cascavel,PR,"[410754, 410500, 410630, 411275, 411670, 41053..."
1,10ª Região,PB,"[250915, 251620, 251398, 251420, 251720, 25084..."
2,10ª Região Limoeiro do Norte,CE,"[230690, 230600, 230760, 231080, 231310, 23115..."
3,10ª Região de Saúde,AL,"[270240, 270330, 270500, 270642, 270580, 27071..."
4,11ª RS Campo Mourão,PR,"[412800, 410755, 411400, 410250, 410770, 41039..."


In [15]:
lista2.head()

,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO
0,Acaraú,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,Acre,AC,"[120001, 120034, 120038, 120040, 120043, 12004..."
2,Afogados da Ingazeira,PE,"[261440, 261153, 260770, 261280, 260390, 26071..."
3,Alagoinhas,BA,"[291650, 291590, 292970, 291370, 291050, 29022..."
4,Alfenas,MG,"[310530, 310410, 314510, 310160, 311440, 31669..."


In [16]:
lista1.shape

(450, 3)

In [17]:
lista2.shape

(358, 3)

In [ ]:
# ================ TESTES=========================

In [18]:
# verificar se tem regiões que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])

In [19]:
df_unicos.shape  #438, 8

(450, 9)

In [20]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], as_index=False).count()

In [21]:
linhas_duplicadas.shape

(450, 9)

In [22]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ATUAL'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas.head()
# Análise: Observando o campo NM_REGIAO_SAUDE_ATUAL e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas. Destaque para a Região de Saúde nomeada de "NORTE". Observando-se RJ e ES, que são contíguos, porém as respectivas Regiões situam-se no Norte de cada Estado.


,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO,CD_REGIAO_SAUDE_2023,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
115,Baixo Amazonas,AM,5,5,5,5,5,5,5
116,Baixo Amazonas,PA,14,14,14,13,13,14,14
145,Central,GO,26,26,26,26,26,26,26
146,Central,RO,14,14,14,14,14,14,14
150,Centro Norte,MT,7,7,7,7,7,7,7


In [23]:
# Até aqui, para a região Norte, não tem Problemas
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Norte']
# Para o caso da Região Baixo Amazonas que abrange 2 UF's, deve-se colocar a condição de agregar, para este caso, as duas UF's 
# teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [24]:
# teste.to_excel('teste2.xlsx')
lista1.to_excel('lista1.xlsx')

In [25]:
teste.shape

(2, 3)

In [26]:
teste.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
277,Norte,GO,"[521377, 521410, 522145, 522020, 520357, 52196..."
278,Norte,RJ,"[330240, 330500, 330475, 330093, 330415, 33048..."


In [27]:
teste['CD_MUNICIPIO'].str.len()

277    13
278     8
Name: CD_MUNICIPIO, dtype: int64

In [28]:
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [29]:
teste['CD_MUNICIPIO'].str.len()

115     5
116    14
Name: CD_MUNICIPIO, dtype: int64

In [30]:
# verificar se tem regiões (ANTERIOR que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])

In [31]:
df_unicos.shape  #362, 9

(362, 9)

In [32]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], as_index=False).count()
linhas_duplicadas.shape # 358, 9

(358, 9)

In [33]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ANTERIOR'], keep=False)]
linhas_duplicadas.shape  #(17, 9)
linhas_duplicadas
# Análise: Observando o campo NM_REGIAO_SAUDE_ANTERIOR e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas.


,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
26,Baixo Amazonas,AM,5,5,5,5,5,5,5
27,Baixo Amazonas,PA,9,9,9,9,9,9,9
187,Metropolitana,PA,5,5,5,5,5,5,5
188,Metropolitana,PR,29,29,29,29,29,29,29
211,Norte,GO,13,13,13,13,13,13,13
212,Norte,RJ,8,8,8,8,8,8,8


In [ ]:
# ================ FIM DOS TESTES =====================

In [34]:
# ========ROTINA INTERMEDIARIA================

# Rotina para agregar a regiao 'Baixo Amazonas' das UF's: AM e PA, por tratar-se de região interestadual. Caso único.
lista_aux1 = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']
lista_aux2 = lista2[lista2['NM_REGIAO_SAUDE_ANTERIOR']=='Baixo Amazonas']


In [35]:
lista_aux1.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
115,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300]"
116,Baixo Amazonas,PA,"[150145, 150050, 150530, 150565, 150600, 15047..."


In [36]:
lista_aux2.head()

,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO
26,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300]"
27,Baixo Amazonas,PA,"[150145, 150050, 150565, 150600, 150285, 15048..."


In [37]:
# uso do drop para SG_UF  para facilitar o gropuby por NM_REGIAO_SAUDE_ATUAL
lista_aux1_new = lista_aux1.drop('SG_UF', axis=1)
lista_aux2_new = lista_aux2.drop('SG_UF', axis=1)

In [38]:
lista_aux1_fim = lista_aux1_new.groupby('NM_REGIAO_SAUDE_ATUAL')['CD_MUNICIPIO'].sum().reset_index()
lista_aux2_fim = lista_aux2_new.groupby('NM_REGIAO_SAUDE_ANTERIOR')['CD_MUNICIPIO'].sum().reset_index()

In [39]:
lista_aux1_fim.head()

,NM_REGIAO_SAUDE_ATUAL,CD_MUNICIPIO
0,Baixo Amazonas,"[130050, 130340, 130290, 130068, 130300, 15014..."


In [40]:
lista_aux2_fim.head()

,NM_REGIAO_SAUDE_ANTERIOR,CD_MUNICIPIO
0,Baixo Amazonas,"[130050, 130340, 130290, 130068, 130300, 15014..."


In [41]:
contar1 = list(lista_aux1_fim['CD_MUNICIPIO'])
# len(lista_aux2_fim['CD_MUNICIPIO'])
# Contar list of values within a pandas df
# contar1 = [sum(a) for a in zip(*lista_aux1_fim['CD_MUNICIPIO'])]
contar1

[[130050,
  130340,
  130290,
  130068,
  130300,
  150145,
  150050,
  150530,
  150565,
  150600,
  150475,
  150285,
  150510,
  150797,
  150480,
  150390,
  150040,
  150300,
  150680]]

In [42]:
len(contar1)

1

In [43]:
teste = lista1.copy()

In [44]:
#atualizacao somente da linha onde está o baixo amazonas. Não esquecer de deletar a outra linha. Baixo Amazonas (PA)
teste.loc[teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas', 'CD_MUNICIPIO'] = contar1

In [45]:
teste = teste.drop(teste[(teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas') & (teste['SG_UF']=='PA')].index) # nao esquecer index
# teste[teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']
# teste.head()

In [46]:
teste[teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
115,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300, 15014..."


In [ ]:
# =========================================
# OPERAÇÕES DE CONCATENAÇÃO DAS LISTAS DE MUNICIPIOS EM LISTA POR REGIOES COM O DATAFRAME ORIGINAL

In [47]:
df_regioes_concat1 = pd.merge(df_regioes, lista1, how = 'left', left_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'], right_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'])
# Agora concatenação com a Regiao anterior, na otica da lista 2
df_regioes_concat2 = pd.merge(df_regioes, lista2, how = 'left', left_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'], right_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'])

In [48]:
df_regioes_concat1.shape

(5571, 10)

In [49]:
df_regioes_concat2.shape

(5571, 10)

In [50]:
df_regioes_concat1.head()

,CD_MUNICIPIO_x,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,CD_MUNICIPIO_y
0,230655,CE,23012,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,230725,CE,23012,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
2,230780,CE,23012,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
3,230020,CE,23012,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
4,230230,CE,23012,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."


In [51]:
df_regioes_concat2.head()

,CD_MUNICIPIO_x,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,CD_MUNICIPIO_y
0,230655,CE,23012,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,230725,CE,23012,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
2,230780,CE,23012,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
3,230020,CE,23012,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
4,230230,CE,23012,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."


In [52]:
# Renomear coluna que foi gerada na juncao do passo anterior
df_regioes_concat1.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_atual'}, inplace=True)
df_regioes_concat2.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_anterior'}, inplace=True)

In [ ]:
# Verificar como os dataframes ficaram apos renomear coluna

In [53]:
df_regioes_concat1.head()

,CD_MUNICIPIO_x,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual
0,230655,CE,23012,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,230725,CE,23012,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
2,230780,CE,23012,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
3,230020,CE,23012,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
4,230230,CE,23012,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."


In [54]:
df_regioes_concat2.head()

,CD_MUNICIPIO_x,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_anterior
0,230655,CE,23012,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,230725,CE,23012,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
2,230780,CE,23012,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
3,230020,CE,23012,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
4,230230,CE,23012,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."


In [55]:
# como são dois dataframes muito identicos em colunas, usei esta técnica para selecionar as do segundo dataframe que sejam diferentes
colunas_mantidas = df_regioes_concat2.columns.difference(df_regioes_concat1.columns)

In [56]:
colunas_mantidas

Index(['lista_regiao_anterior'], dtype='object')

In [57]:
df_regioes_concat = pd.merge(df_regioes_concat1, df_regioes_concat2[colunas_mantidas], left_index=True, right_index=True, how='outer')

In [58]:
df_regioes_concat.rename(columns={'CD_MUNICIPIO_x':'CD_MUNICIPIO'}, inplace=True)

In [59]:
df_regioes_concat['flag_atual'] = np.where(df_regioes_concat['lista_regiao_atual'] == df_regioes_concat['lista_regiao_anterior'], 0, 1)

In [60]:
df_regioes_concat.head()

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual
0,230655,CE,23012,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
1,230725,CE,23012,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
2,230780,CE,23012,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
3,230020,CE,23012,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
4,230230,CE,23012,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0


In [ ]:
# ============ PASSO PARA MERGE COM A TABLE DE MUNICIPIOS PARA AGREGAR CAMPOS DESTA TABELA

In [61]:
df_municipios = pd.read_excel('AR_BR_RG_UF_MES_MIC_MUN_2018.xls')

In [62]:
df_municipios.head()

,ID,CD_GCUF,NM_UF,NM_UF_SIGLA,CD_GCMUN,NM_MUN_2018,AR_MUN_2018
0,1.0,11.0,Rondônia,RO,1100015.0,ALTA FLORESTA D'OESTE,7067.025
1,2.0,11.0,Rondônia,RO,1100023.0,ARIQUEMES,4426.571
2,3.0,11.0,Rondônia,RO,1100031.0,CABIXI,1314.352
3,4.0,11.0,Rondônia,RO,1100049.0,CACOAL,3792.892
4,5.0,11.0,Rondônia,RO,1100056.0,CEREJEIRAS,2783.300


In [63]:
df_municipios.dropna(axis=0, how='any', subset=['CD_GCMUN'], inplace=True)

In [64]:
df_municipios['CD_MUNICIPIO_NDV'] = df_municipios['CD_GCMUN'].astype(str).str[:6]

In [65]:
type(df_municipios['CD_MUNICIPIO_NDV'][0])

str

In [66]:
df_regioes_concat['CD_MUNICIPIO'] = df_regioes_concat['CD_MUNICIPIO'].astype(str)

In [67]:
df_diferenca = pd.merge(df_regioes_concat, df_municipios, how = 'outer', left_on='CD_MUNICIPIO', right_on='CD_MUNICIPIO_NDV')

In [68]:
df_diferenca_qgis = df_diferenca[df_diferenca['flag_atual']==1]

In [69]:
df_regioes_concat.rename(columns={'SG_UF_x':'SG_UF'}, inplace=True)

In [70]:
df_regioes_concat.to_excel('teste.xlsx')

In [71]:
df_regioes_concat.head()

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual
0,230655,CE,23012,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
1,230725,CE,23012,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
2,230780,CE,23012,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
3,230020,CE,23012,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
4,230230,CE,23012,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0


In [72]:
resumo1 = df_regioes_concat[['SG_UF', 'NM_MUNICIPIO', 'NM_REGIAO_SAUDE_ATUAL']]

In [73]:
df_reg_mesmos_municipios = df_regioes_concat[df_regioes_concat['flag_atual']==0]

In [74]:
df_reg_mesmos_municipios.shape

(2101, 12)

In [75]:
municipios_sem_mudar_regiao = df_reg_mesmos_municipios.shape[0]

In [76]:
municipios_sem_mudar_regiao

2101

In [77]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ATUAL').count()

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual
NM_REGIAO_SAUDE_ATUAL,,,,,,,,,,,
10ª RS Cascavel,25,25,25,25,25,25,25,25,25,25,25
10ª Região Limoeiro do Norte,11,11,11,11,11,11,11,11,11,11,11
11ª RS Campo Mourão,25,25,25,25,25,25,25,25,25,25,25
11ª Região,7,7,7,7,7,7,7,7,7,7,7
11ª Região Sobral,24,24,24,24,24,24,24,24,24,24,24
...,...,...,...,...,...,...,...,...,...,...,...
Vale dos Arinos,4,4,4,4,4,4,4,4,4,4,4
Vale dos Rios Piauí e Itaueiras,28,28,28,28,28,28,28,28,28,28,28
Vitória da Conquista,19,19,19,19,19,19,19,19,19,19,19


In [78]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ANTERIOR').count()

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual
NM_REGIAO_SAUDE_ANTERIOR,,,,,,,,,,,
Acaraú,7,7,7,7,7,7,7,7,7,7,7
Afogados da Ingazeira,12,12,12,12,12,12,12,12,12,12,12
Alagoinhas,18,18,18,18,18,18,18,18,18,18,18
Alta Floresta,6,6,6,6,6,6,6,6,6,6,6
Apucarana,17,17,17,17,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...,...,...,...
Vitória da Conquista,19,19,19,19,19,19,19,19,19,19,19
XI Núcleo Regional de Saúde,7,7,7,7,7,7,7,7,7,7,7
XII Núcleo Regional de Saúde,14,14,14,14,14,14,14,14,14,14,14


In [79]:
resumo2 = df_reg_mesmos_municipios.groupby(['NM_REGIAO_SAUDE_ATUAL','NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).count()['CD_MUNICIPIO'].reset_index()

In [80]:
resumo2.sort_values('CD_MUNICIPIO', ascending=False, inplace=True)

In [81]:
resumo2

,NM_REGIAO_SAUDE_ATUAL,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO
147,Vale do Rio Guaribas,TD Vale do Rio Guaribas,PI,42
42,6ª Região de Saúde - Pau dos Ferros,VI ETAM,RN,37
64,Caruaru,Caruaru,PE,32
74,Entre Rios,TD Entre Rios,PI,31
12,15ª RS Maringá,Maringá,PR,30
...,...,...,...,...
149,Vale dos Arinos,Juara,MT,4
11,14ª Região Tauá,Tauá,CE,4
58,Baia da Ilha Grande,Baía da Ilha Grande,RJ,3
70,Corumbá,NCT de Corumbá,MS,2


In [82]:
df_regioes_concat.sort_values('SG_UF', ascending=True, inplace=True)

In [83]:
for row in df_regioes_concat.loc[df_regioes_concat['lista_regiao_anterior'].isnull(), 'lista_regiao_anterior'].index:
    df_regioes_concat.at[row, 'lista_regiao_anterior'] = []

In [84]:
df_regioes_concat[df_regioes_concat['lista_regiao_anterior'].isnull()]

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual


In [85]:
df_regioes_concat['flag_municipio_entrou'] = df_regioes_concat['lista_regiao_atual'].map(set) - df_regioes_concat['lista_regiao_anterior'].map(set)

In [86]:
df_regioes_concat['flag_municipio_saiu'] = df_regioes_concat['lista_regiao_anterior'].map(set) - df_regioes_concat['lista_regiao_atual'].map(set)

In [87]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Baixo Acre e Purus']

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu
1497,120032,AC,12002,Baixo Acre e Purus,Jordão,Cruzeiro do Sul,Jordão,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120032, 120033, 120035, 120039, 120042, 12002...",1,"{120001, 120034, 120038, 120040, 120043, 12004...","{120033, 120035, 120039, 120042, 120020, 12006..."
15,120001,AC,12002,Baixo Acre e Purus,Acrelândia,Acre,Acrelândia,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
14,120013,AC,12002,Baixo Acre e Purus,Bujari,Acre,Bujari,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
13,120017,AC,12002,Baixo Acre e Purus,Capixaba,Acre,Capixaba,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
16,120080,AC,12002,Baixo Acre e Purus,Porto Acre,Acre,Porto Acre,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
11,120038,AC,12002,Baixo Acre e Purus,Plácido de Castro,Acre,Plácido de Castro,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
12,120034,AC,12002,Baixo Acre e Purus,Manoel Urbano,Acre,Manoel Urbano,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
8,120050,AC,12002,Baixo Acre e Purus,Sena Madureira,Acre,Sena Madureira,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
9,120043,AC,12002,Baixo Acre e Purus,Santa Rosa do Purus,Acre,Santa Rosa do Purus,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
10,120040,AC,12002,Baixo Acre e Purus,Rio Branco,Acre,Rio Branco,ACRE,NORTE,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}


In [88]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Norte']

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu
3201,521525,GO,52008,Norte,Novo Planalto,Norte,Novo Planalto,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3200,522145,GO,52008,Norte,Trombas,Norte,Trombas,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3199,521377,GO,52008,Norte,Montividiu do Norte,Norte,Montividiu do Norte,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3198,521405,GO,52008,Norte,Mundo Novo,Norte,Mundo Novo,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3197,521410,GO,52008,Norte,Mutunópolis,Norte,Mutunópolis,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3196,520357,GO,52008,Norte,Bonópolis,Norte,Bonópolis,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3195,521308,GO,52008,Norte,Minaçu,Norte,Minaçu,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3194,521800,GO,52008,Norte,Porangatu,Norte,Porangatu,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3193,522020,GO,52008,Norte,São Miguel do Araguaia,Norte,São Miguel do Araguaia,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}
3189,521960,GO,52008,Norte,Santa Tereza de Goiás,Norte,Santa Tereza de Goiás,GOIÁS,CENTRO-OESTE,"[521377, 521410, 522145, 522020, 520357, 52196...","[521377, 521410, 522145, 522020, 520357, 52196...",0,{},{}


In [89]:
df_regioes_concat['cont_municipio_entrou'] = [len(x) for x in df_regioes_concat['flag_municipio_entrou']]

In [90]:
df_regioes_concat['cont_municipio_saiu'] = [len(x) for x in df_regioes_concat['flag_municipio_saiu']]

In [91]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_entrou']>1)]

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu,cont_municipio_entrou,cont_municipio_saiu


In [92]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_saiu']>1)]

,CD_MUNICIPIO,SG_UF,CD_REGIAO_SAUDE_2023,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu,cont_municipio_entrou,cont_municipio_saiu


In [93]:
df_estatistica_entrada = df_regioes_concat[(df_regioes_concat['flag_atual']==1)&(df_regioes_concat['cont_municipio_entrou']!=0)]

In [94]:
df_estatistica_entrada.shape

(1412, 16)

In [95]:
df_result_atual = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ATUAL'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

C:\Users\domdo\AppData\Local\Temp\ipykernel_12316\760642781.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_result_atual = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ATUAL'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()


In [96]:
df_result_atual

SG_UF       NM_REGIAO_SAUDE_ATUAL cont_municipio_entrou                     \
                                                     count min max       mean   
0     AC          Baixo Acre e Purus                    11   1  10   1.818182   
1     AL          1ª Região de Saúde                    12   1  11   1.833333   
2     AL          4ª Região de Saúde                     9   2   7   3.111111   
3     AL          7ª Região de Saúde                    17   3  14   4.941176   
4     AM               Alto Solimões                     9   2   7   3.111111   
..   ...                         ...                   ...  ..  ..        ...   
89    TO               Capim Dourado                    14   6  13   7.571429   
90    TO  Cerrado Tocantins Araguaia                    23  13  17  14.956522   
91    TO             Ilha do Bananal                    18   5  13   7.222222   
92    TO        Médio Norte Araguaia                    17   4  16   6.470588   
93    TO                     Sudeste                    15   7   8   7.466667   

   cont_municipio_saiu                     
                 count min max       mean  
0                   11   0   7   0.636364  
1                   12  11  16  15.583333  
2                    9  20  22  20.444444  
3                   17  10  21  11.941176  
4                    9   0  10   2.222222  
..                 ...  ..  ..        ...  
89                  14   0   7   0.500000  
90                  23   0   1   0.304348  
91                  18   0   0   0.000000  
92                  17   0   6   1.058824  
93                  15   0   0   0.000000  

[94 rows x 10 columns]

In [97]:
df_result_anterior = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ANTERIOR'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

C:\Users\domdo\AppData\Local\Temp\ipykernel_12316\299815066.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_result_anterior = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ANTERIOR'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()


In [98]:
df_result_anterior.head()

SG_UF NM_REGIAO_SAUDE_ANTERIOR cont_municipio_entrou                     \
                                                 count min max       mean   
0    AC                     Acre                    10   1   1   1.000000   
1    AC          Cruzeiro do Sul                     1  10  10  10.000000   
2    AL                Região 01                    18   1   2   1.388889   
3    AL                Região 03                     3  14  14  14.000000   
4    AL                Região 04                    16   3   7   3.500000   

  cont_municipio_saiu                     
                count min max       mean  
0                  10   0   0   0.000000  
1                   1   7   7   7.000000  
2                  18  16  20  17.555556  
3                   3  21  21  21.000000  
4                  16  10  22  11.500000

In [99]:
df_result_atual.to_excel('resultado_atual.xlsx')

In [100]:
df_result_anterior.to_excel('resultado_anterior.xlsx')

In [101]:
df_estatistica_entrada.to_excel('resumo_geral_movimentacao.xlsx')

In [ ]:
# Sum list of values within a pandas df
# df = [sum(a) for a in zip(*df['Val'])]